In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from pyspark.sql.functions import *

In [0]:
%sql
use catalog `pricing-analytics`;
create database if not exists data;

In [0]:
%sql
-- truncate table `pricing-analytics`.data.date_processed

In [0]:
%sql
create table if not exists `pricing-analytics`.data.date_processed(
  date date 
)

In [0]:
dbutils.widgets.text("processed_date", "")

In [0]:
processed_date = dbutils.widgets.get('processed_date')

In [0]:

try:
  processed_date = datetime.strptime(processed_date, '%Y-%m-%d').date()
except:
  processed_date = processed_date
x=True
while x == True:
  df = spark.read.table("`pricing-analytics`.data.date_processed")
  date_list = df.select("date").rdd.flatMap(lambda x: x).collect()
  if processed_date in date_list:
    print("Data already processed")
    x = False
  else:
    try:
        filename = processed_date.strftime("%d%m%Y")
        df1 = pd.read_csv(f"https://retailpricing.blob.core.windows.net/daily-pricing/PW_MW_DR_{filename}.csv")
        df1 = spark.createDataFrame(df1)
        df1.repartition(1).write.mode("append").format('csv').save(f'abfss://pricing-analytics@adlsudemycourse1.dfs.core.windows.net/pricing_files')
        spark.sql(f"""insert into `pricing-analytics`.data.date_processed values (cast('{processed_date}' as date))""")
        print(f"data processed for : {processed_date}")
        sys.stdout.flush() 
        processed_date = processed_date + timedelta(days=1)
    except Exception as e:
      x=False
      print("Data not available")



In [0]:
# dbutils.fs.rm('abfss://pricing-analytics@adlsudemycourse1.dfs.core.windows.net/pricing_files/',True)

In [0]:
%sql
-- drop table `pricing-analytics`.data.combines

In [0]:
%sql
create external table `pricing-analytics`.data.combines(
    DATE_OF_PRICING   varchar(50),
    ROW_ID  varchar(50),
    STATE_NAME varchar(50),
    MARKET_NAME varchar(50),
    PRODUCTGROUP_NAME varchar(50),
    PRODUCT_NAME   varchar(50),
    VARIETY   varchar(50),
    ORIGIN   varchar(50),
    ARRIVAL_IN_TONNES varchar(50),
    MINIMUM_PRICE   varchar(50),
    MAXIMUM_PRICE   varchar(50),
    MODAL_PRICE  varchar(50)
)
using csv
Location 'abfss://pricing-analytics@adlsudemycourse1.dfs.core.windows.net/pricing_files/'

